Import

In [258]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import math

Load necessary data

In [259]:
apple = yf.Ticker("AAPL")
apple = apple.history(start="1981-01-01", end="2023-12-31", auto_adjust=True)
ms = yf.Ticker("MSFT")
ms = ms.history(start="1981-01-01", end="2023-12-31", auto_adjust=True)
nvd = yf.Ticker("NVDA")
nvd = nvd.history(start="1981-01-01", end="2023-12-31", auto_adjust=True)
google = yf.Ticker("GOOG")
google = google.history(start="1981-01-01", end="2023-12-31", auto_adjust=True)
meta = yf.Ticker("META")
meta = meta.history(start="1981-01-01", end="2023-12-31", auto_adjust=True)
tsla = yf.Ticker("TSLA")
tsla = tsla.history(start="1981-01-01", end="2023-12-31", auto_adjust=True)
amz = yf.Ticker("AMZN")
amz = amz.history(start="1981-01-01", end="2023-12-31", auto_adjust=True)

In [253]:
print(apple.columns)
print(apple.head())
print(len(apple))
print(len(ms))
print(len(tsla))
print(len(amz))
print(len(meta))
print(len(google))
print(len(nvd))

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits'],
      dtype='object')
         Date      Open      High       Low     Close    Volume  Dividends  \
0  1981-01-02  0.118471  0.119330  0.118471  0.118471  21660800        0.0   
1  1981-01-05  0.116325  0.116325  0.115896  0.115896  35728000        0.0   
2  1981-01-06  0.111174  0.111174  0.110745  0.110745  45158400        0.0   
3  1981-01-07  0.106453  0.106453  0.106023  0.106023  55686400        0.0   
4  1981-01-08  0.104307  0.104307  0.103877  0.103877  39827200        0.0   

   Stock Splits  
0           0.0  
1           0.0  
2           0.0  
3           0.0  
4           0.0  
10840
9527
3400
6701
2923
4875
6276


In [260]:
#Align the dates of all the datasets
def align_dataframes(apple, google, meta, amz, tsla, ms, nvd):
    dfs = [apple, google, meta, amz, tsla, ms, nvd]

    for df in dfs:
        if 'Date' in df.columns:
            if not pd.api.types.is_datetime64_any_dtype(df['Date']):
                df['Date'] = pd.to_datetime(df['Date'])

    for df in dfs:
        if 'Date' in df.columns and df.index.name != 'Date':
            df.set_index('Date', inplace=True)

    common_dates = dfs[0].index
    for df in dfs[1:]:
        common_dates = common_dates.intersection(df.index)

    aligned_dfs = []
    for df in dfs:
        aligned_dfs.append(df.loc[common_dates].copy())

    for df in aligned_dfs:
        df.reset_index(inplace=True)

    return aligned_dfs

In [261]:
dfs = align_dataframes(apple, google, meta, amz, tsla, ms, nvd)
apple, google, meta, amz, tsla, ms, nvd = align_dataframes(apple, google, meta, amz, tsla, ms, nvd)

In [263]:
apple = apple.reset_index()
apple['Date'] = apple['Date'].dt.date

In [264]:
ms = ms.reset_index()
ms['Date'] = ms['Date'].dt.date
print(ms.head())

AttributeError: Can only use .dt accessor with datetimelike values

In [265]:
meta = meta.reset_index()
meta['Date'] = meta['Date'].dt.date
print(meta.head())

   index        Date       Open       High        Low      Close     Volume  \
0      0  2012-05-18  41.852747  44.788910  37.821746  38.050667  573576400   
1      1  2012-05-21  36.358646  36.488037  32.845205  33.870373  168192700   
2      2  2012-05-22  32.457030  33.432433  30.794864  30.854582  101786600   
3      3  2012-05-23  31.222848  32.347546  31.212894  31.849892   73600000   
4      4  2012-05-24  32.795438  33.054217  31.620973  32.875061   50237200   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  


In [266]:
amz = amz.reset_index()
amz['Date'] = amz['Date'].dt.date
print(amz.head())

google = google.reset_index()
google['Date'] = google['Date'].dt.date
print(google.head())

nvd = nvd.reset_index()
nvd['Date'] = nvd['Date'].dt.date
print(nvd.head())

tsla = tsla.reset_index()
tsla['Date'] = tsla['Date'].dt.date
print(tsla.head())

   index        Date     Open     High      Low    Close     Volume  \
0      0  2012-05-18  10.9705  10.9815  10.6405  10.6925  104634000   
1      1  2012-05-21  10.7015  10.9990  10.6410  10.9055   71596000   
2      2  2012-05-22  10.9155  10.9435  10.6980  10.7665   74662000   
3      3  2012-05-23  10.7355  10.8775  10.5590  10.8640   84876000   
4      4  2012-05-24  10.8490  10.8830  10.6350  10.7620   62822000   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  
   index        Date       Open       High        Low      Close     Volume  \
0      0  2012-05-18  15.495749  15.677206  14.791734  14.883454  239835606   
1      1  2012-05-21  14.886181  15.262482  14.873538  15.223314  123477094   
2      2  2012-05-22  15.206706  15.215877  14.774382  14.893370  122533571   
3      3  2012-05-23  14.914440  15.111515  14.802145  15.108044  127600492   
4   

RSI Indicator

In [267]:
#time lag
def rsi_indicator(df, wd):
    closed = df['Close']
    delta = closed.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=wd).mean()
    avg_loss = loss.rolling(window=wd).mean()
    rs = avg_gain/avg_loss
    rsi = 100 - (100 / (1 + rs))
    df['RSI'] = rsi
    return df


In [268]:
rsi_indicator(apple, 14)
rsi_indicator(ms, 14)
rsi_indicator(meta, 14)
rsi_indicator(nvd, 14)
rsi_indicator(tsla, 14)
rsi_indicator(amz, 14)
rsi_indicator(google, 14)
print(apple.head())

   index        Date       Open       High        Low      Close     Volume  \
0      0  2012-05-18  16.068769  16.353153  15.714267  15.961034  732292400   
1      1  2012-05-21  16.085031  16.898761  16.071487  16.890936  631106000   
2      2  2012-05-22  17.139806  17.270112  16.629119  16.761230  694870400   
3      3  2012-05-23  16.777180  17.237614  16.648682  17.170204  584897600   
4      4  2012-05-24  17.329997  17.348957  16.889427  17.012510  496230000   

   Dividends  Stock Splits  RSI  
0        0.0           0.0  NaN  
1        0.0           0.0  NaN  
2        0.0           0.0  NaN  
3        0.0           0.0  NaN  
4        0.0           0.0  NaN  


In [269]:
def rsi_signal(df):
    df['Signal'] = df['RSI'].apply(lambda x: 'Buy' if x < 25 else ('Sell' if x > 75 else 'Hold')).shift(1)
    return df 

In [270]:
rsi_signal(apple)
rsi_signal(ms)
rsi_signal(meta)
rsi_signal(amz)
rsi_signal(nvd)
rsi_signal(google)
rsi_signal(tsla)

index        Date        Open        High         Low       Close  \
0         0  2012-05-18    1.891333    1.897333    1.788667    1.837333   
1         1  2012-05-21    1.838667    1.950667    1.808000    1.918000   
2         2  2012-05-22    2.006667    2.089333    2.000000    2.053333   
3         3  2012-05-23    2.037333    2.070000    1.966667    2.068000   
4         4  2012-05-24    2.083333    2.083333    1.979333    2.018667   
...     ...         ...         ...         ...         ...         ...   
2918   2918  2023-12-22  256.760010  258.220001  251.369995  252.539993   
2919   2919  2023-12-26  254.490005  257.970001  252.910004  256.609985   
2920   2920  2023-12-27  258.350006  263.339996  257.519989  261.440002   
2921   2921  2023-12-28  263.660004  265.130005  252.710007  253.179993   
2922   2922  2023-12-29  255.100006  255.190002  247.429993  248.479996   

         Volume  Dividends  Stock Splits        RSI Signal  
0      24247500        0.0           0.0        NaN   None  
1      22128000        0.0           0.0        NaN   Hold  
2      35493000        0.0           0.0        NaN   Hold  
3      18306000        0.0           0.0        NaN   Hold  
4      16134000        0.0           0.0        NaN   Hold  
...         ...        ...           ...        ...    ...  
2918   93249800        0.0           0.0  64.737564   Hold  
2919   86892400        0.0           0.0  65.298430   Hold  
2920  106494400        0.0           0.0  67.613726   Hold  
2921  113619900        0.0           0.0  57.791239   Hold  
2922  100615300        0.0           0.0  53.261173   Hold  

[2923 rows x 11 columns]

Portfolio Allocation

Bought on the criterion of market conditions
Assume if exceed capital to buy min stocks that meet requirement, don't buy


In [271]:
cap = 10000  # Initial capital
slip = 1.001  # Slippage factor
com = 1.002  # Commission factor
risk_free = 0.02 #Risk free rate per annum
previous_portfolio_value = 0
trading_days = 252
trades_made = 0

#wr
win_count = 0
loss_count = 0
total_trades = 0

#expectancy
gains = []
losses = []

In [273]:
pf = pd.DataFrame(columns=['Date', 'APPL', 'GOOG', 'META', 'AMZN', 'TSLA', 'MSFT', 'NVDA', 'Capital', 'Value', 'Returns', 'APPL_Shares', 'GOOG_Shares', 'META_Shares', 'AMZN_Shares', 'TSLA_Shares', 'MSFT_Shares', 'NVDA_Shares', 'Stocks_Bought', 'Daily_Return'])

#num of shares for each stock
shares = {
    'APPL': 0,
    'GOOG': 0,
    'META': 0,
    'AMZN': 0,
    'TSLA': 0,
    'MSFT': 0,
    'NVDA': 0
}

# Track previous signals to detect changes
prev_signals = {
    'APPL': None,
    'GOOG': None,
    'META': None,
    'AMZN': None,
    'TSLA': None,
    'MSFT': None,
    'NVDA': None
}

for i in range(len(apple)):
    # Get current prices for all stocks
    prices = {
        'APPL': apple.iloc[i]['Open'],
        'GOOG': google.iloc[i]['Open'],
        'META': meta.iloc[i]['Open'],
        'AMZN': amz.iloc[i]['Open'],
        'TSLA': tsla.iloc[i]['Open'],
        'MSFT': ms.iloc[i]['Open'],
        'NVDA': nvd.iloc[i]['Open']
    }

    # Get current signals
    current_signals = {
        'APPL': apple.iloc[i]['Signal'],
        'GOOG': google.iloc[i]['Signal'],
        'META': meta.iloc[i]['Signal'],
        'AMZN': amz.iloc[i]['Signal'],
        'TSLA': tsla.iloc[i]['Signal'],
        'MSFT': ms.iloc[i]['Signal'],
        'NVDA': nvd.iloc[i]['Signal']
    }

    # current portfolio value (AUM)
    portfolio_value = cap
    for stock, share_count in shares.items():
        portfolio_value += share_count * prices[stock]

    # Check rebalancing
    for stock in shares.keys():
        stock_value = shares[stock] * prices[stock]
        stock_percentage = stock_value / portfolio_value if portfolio_value > 0 else 0

        if stock_percentage > 0.3 and prev_signals[stock] is not None and current_signals[stock] != prev_signals[stock]:
            target_value = portfolio_value * 0.3
            shares_to_sell = math.ceil((stock_value - target_value) / prices[stock])
            shares_to_sell = min(shares_to_sell, shares[stock])

            shares[stock] -= shares_to_sell
            cap += shares_to_sell * prices[stock] / (slip * com) 
            trades_made += 1
            print(f"Rebalancing: Sold {shares_to_sell} shares of {stock} at {prices[stock]}")

    prev_signals = current_signals.copy()

    stocks_to_buy = []
    Open = 0

    # Check buy signals
    for stock, signal in current_signals.items():
        if signal == 'Buy':
            Open += prices[stock] * slip * com
            stocks_to_buy.append(stock)

    # stocks bought today and num of shares
    stocks_bought_today = {}

    # shares to buy
    base_entry = Open * 10
    num_shares_to_buy = 0

    if base_entry > 0: 
        if base_entry > cap:
            print(f"Not enough capital for {apple.iloc[i]['Date']}")
        else:
            diff = cap - base_entry
            if diff > Open:
                extra = math.floor(diff/Open)
                num_shares_to_buy = 10 + extra
            else:
                num_shares_to_buy = 10

            #Equally weighted portfolio allocation
            if stocks_to_buy:
                shares_per_stock = math.floor(num_shares_to_buy / len(stocks_to_buy))
                for stock in stocks_to_buy:
                    shares[stock] += shares_per_stock
                    cap -= shares_per_stock * prices[stock] * slip * com
                    stocks_bought_today[stock] = shares_per_stock
                    trades_made += len(stocks_bought_today)

    # current total portfolio value
    new_portfolio_value = cap
    for stock, share_count in shares.items():
        new_portfolio_value += share_count * prices[stock]

    #daily return
    daily_return = 0
    win_loss = 'No Trade'
    if previous_portfolio_value > 0:
        daily_return = (new_portfolio_value - previous_portfolio_value) / previous_portfolio_value
        
        if daily_return > 0:
            win_loss = 'Win'
            win_count += 1
            gains.append(daily_return)
            total_trades += 1
        elif daily_return < 0:
            win_loss = 'Loss'
            loss_count += 1
            losses.append(daily_return)
            total_trades += 1
        else:
            win_loss = 'Flat'

    # cumulative returns
    cumulative_returns = 0
    if len(pf) > 0:
        try:
            first_value = pf['Value'].iloc[0]
            if first_value > 0:
                cumulative_returns = (new_portfolio_value - first_value) / first_value
        except IndexError:
            cumulative_returns = 0

    stocks_bought_str = ", ".join([f"{stock}: {shares}" for stock, shares in stocks_bought_today.items()]) if stocks_bought_today else "None"

    # new row
    new_row = pd.DataFrame({
        'Date': [apple.iloc[i]['Date']],
        'APPL': [prices['APPL']],
        'GOOG': [prices['GOOG']],
        'META': [prices['META']],
        'AMZN': [prices['AMZN']],
        'TSLA': [prices['TSLA']],
        'MSFT': [prices['MSFT']],
        'NVDA': [prices['NVDA']],
        'Capital': [cap],
        'Value': [new_portfolio_value],
        'Returns': [cumulative_returns],
        'APPL_Shares': [shares['APPL']],
        'GOOG_Shares': [shares['GOOG']],
        'META_Shares': [shares['META']],
        'AMZN_Shares': [shares['AMZN']],
        'TSLA_Shares': [shares['TSLA']],
        'MSFT_Shares': [shares['MSFT']],
        'NVDA_Shares': [shares['NVDA']],
        'Stocks_Bought': [stocks_bought_str],
        'Daily_Return': [daily_return]
    })
    
    previous_portfolio_value = new_portfolio_value
    pf = pd.concat([pf, new_row], ignore_index=True)

    #Win Rate
    win_rate = win_count / total_trades if total_trades > 0 else 0
    
    #Expectancy
    avg_gain = sum(gains) / len(gains) if gains else 0
    avg_loss = sum(losses) / len(losses) if losses else 0
    expectancy = (win_rate * avg_gain) + ((1 - win_rate) * avg_loss) if total_trades > 0 else 0
print(pf[['Date', 'Stocks_Bought', 'Daily_Return', 'Value', 'Returns']].head(20))

def print_portfolio_summary():
    print("\nPortfolio Summary:")
    print("-----------------")
    latest = pf.iloc[-1]
    total_value = latest['Value']

    print(f"Date: {latest['Date']}")
    print(f"Total Portfolio Value: ${total_value:.2f}")
    print(f"Cash: ${latest['Capital']:.2f}")
    print(f"Cumulative Return: {latest['Returns']:.2%}")
    print(f"Total Number of Trades :{trades_made}")
    print(f"Win Rate :{win_rate:.2%}")
    print(f"Expectancy :{expectancy}")

    print("\nStock Holdings:")
    for stock in ['APPL', 'GOOG', 'META', 'AMZN', 'TSLA', 'MSFT', 'NVDA']:
        shares = latest[f'{stock}_Shares']
        price = latest[stock]
        value = shares * price
        allocation = value / total_value if total_value > 0 else 0

        if shares > 0:
            print(f"{stock}: {shares} shares @ ${price:.2f} = ${value:.2f} ({allocation:.2%} of portfolio)")

print_portfolio_summary()

/var/folders/1d/304spk_544j6rfmzkcz0plcr0000gn/T/ipykernel_71450/2778412807.py:167: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pf = pd.concat([pf, new_row], ignore_index=True)


Rebalancing: Sold 224 shares of META at 27.351091855812506
Rebalancing: Sold 249 shares of GOOG at 13.890397449090743
Rebalancing: Sold 11 shares of META at 32.70585921477661
Rebalancing: Sold 125 shares of META at 29.272042596809303
Not enough capital for 2012-07-30
Rebalancing: Sold 20 shares of GOOG at 15.57408225516286
Not enough capital for 2012-08-01
Not enough capital for 2012-08-02
Not enough capital for 2012-08-03
Not enough capital for 2012-08-06
Rebalancing: Sold 149 shares of META at 22.095863607733662
Not enough capital for 2012-08-09
Rebalancing: Sold 8 shares of GOOG at 15.830155203128724
Not enough capital for 2012-08-10
Rebalancing: Sold 155 shares of META at 22.04609552940136
Rebalancing: Sold 1 shares of GOOG at 16.342299020423358
Rebalancing: Sold 166 shares of META at 20.344118405884917
Rebalancing: Sold 8 shares of GOOG at 16.484838646846697
Rebalancing: Sold 2 shares of GOOG at 16.96178132600879
Rebalancing: Sold 2 shares of GOOG at 16.955833627608683
Rebalancing

In [274]:
# Consider in the risk-free annum
if len(pf) > 0:

    total_days = len(pf)
    years = total_days / trading_days
    final_value = pf.iloc[-1]['Value']
    initial_value = pf.iloc[0]['Value'] if 'Value' in pf.iloc[0] and pf.iloc[0]['Value'] > 0 else 10000

    #cumulative return
    cumulative_return = (final_value / initial_value) - 1
    annual_return = (1 + cumulative_return) ** (1 / years) - 1
    daily_risk_free = (1 + risk_free) ** (1 / trading_days) - 1

    #risk-free return
    cumulative_risk_free = (1 + risk_free) ** years - 1
    #excess return
    excess_return = cumulative_return - cumulative_risk_free

    annual_excess_return = annual_return - risk_free

    # Volatility
    pf['log_return'] = np.log(pf['Value'] / pf['Value'].shift(1))
    daily_volatility = pf['log_return'].std()
    annual_volatility = daily_volatility * np.sqrt(trading_days)

    #Maximum drawdown
    pf['cumulative_return'] = (pf['Value'] / initial_value) - 1
    pf['cumulative_max'] = pf['cumulative_return'].cummax()
    pf['drawdown'] = (pf['cumulative_return'] - pf['cumulative_max']) / (1 + pf['cumulative_max'])
    max_drawdown = pf['drawdown'].min()

    #Sharpe ratio
    sharpe_ratio = annual_excess_return / annual_volatility if annual_volatility > 0 else 0

    #Sortino ratio
    pf['daily_return'] = pf['Value'] / pf['Value'].shift(1) - 1
    pf['excess_daily_return'] = pf['daily_return'] - daily_risk_free
    downside_returns = pf['excess_daily_return'][pf['excess_daily_return'] < 0]
    downside_deviation = downside_returns.std() * np.sqrt(trading_days) if len(downside_returns) > 0 else 0
    sortino_ratio = annual_excess_return / downside_deviation if downside_deviation != 0 else float('nan')

    # Average return per trade
    average_return_per_trade = cumulative_return / trades_made if trades_made > 0 else 0

    print("\nPerformance Metrics:")
    print("-------------------")
    print(f"Total Returns: {cumulative_return:.2%}")
    print(f"Annual Return: {annual_return:.2%}")
    print(f"Annual Volatility: {annual_volatility:.2%}")
    print(f"Maximum Drawdown: {max_drawdown:.2%}")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Sortino Ratio: {sortino_ratio:.2f}")
    print(f"Average Return per Trade: {average_return_per_trade:.2%}")


Performance Metrics:
-------------------
Total Returns: 4174.08%
Annual Return: 38.23%
Annual Volatility: 30.50%
Maximum Drawdown: -52.30%
Sharpe Ratio: 1.19
Sortino Ratio: 1.57
Average Return per Trade: 11.28%


In [275]:
#Question 1
date = pd.to_datetime(pf.loc[pf['Returns'].idxmax()]['Date'])
print("Month of highest returns is", date.strftime('%B'))

allocations = {}
row= pf.loc[pf['Returns'].idxmax()]
for stock in ['APPL', 'GOOG', 'META', 'AMZN', 'TSLA', 'MSFT', 'NVDA']:
    shares = row[f'{stock}_Shares']
    price = row[stock]
    value = shares * price
    allocation = value / row['Value'] if row['Value'] > 0 else 0
    allocations[stock] = allocation
    
stock_name = max(allocations, key=allocations.get)
print(f"Stock of highest contribution is {stock_name} with allocation {allocations[stock_name]:.2%}")

Month of highest returns is December
Stock of highest contribution is NVDA with allocation 28.75%


In [276]:
#Question 2
sp500 = yf.download('^GSPC', start="1981-01-01", end="2023-12-31", auto_adjust = True)
print(sp500.head())
print(sp500.columns)

[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low  Open    Volume
Ticker           ^GSPC       ^GSPC       ^GSPC ^GSPC     ^GSPC
Date                                                          
1981-01-02  136.339996  137.100006  134.610001   0.0  28870000
1981-01-05  137.970001  139.240005  135.860001   0.0  58710000
1981-01-06  138.119995  140.320007  135.779999   0.0  67400000
1981-01-07  135.080002  136.020004  132.300003   0.0  92890000
1981-01-08  133.059998  136.100006  131.960007   0.0  55350000
MultiIndex([( 'Close', '^GSPC'),
            (  'High', '^GSPC'),
            (   'Low', '^GSPC'),
            (  'Open', '^GSPC'),
            ('Volume', '^GSPC')],
           names=['Price', 'Ticker'])


In [277]:
#match dates
pf = pf.reset_index()
pf['Date'] = pd.to_datetime(pf['Date'])
pf.set_index('Date', inplace=True)
sp500 = sp500['Close', '^GSPC']

pf['SP'] = sp500

# S&P 500 returns
pf['SP_Return'] = pf['SP'].pct_change()
pf['SP_Cum_Return'] = (1 + pf['SP_Return']).cumprod() - 1

# Comparison
pf_return = pf['Returns'].iloc[-1]
sp_return = pf['SP_Cum_Return'].iloc[-1]
print(f"Portfolio Return: {pf_return:.2%}")
print(f"S&P 500 Return: {sp_return:.2%}")

Portfolio Return: 4174.08%
S&P 500 Return: 268.26%


Portfolio returns outperform S&P 500 returns by a large margin. I am mainly comparing the final cumulative returns if the backtesting strategy is used compared to the returns of the S&P 500. The difference in return margin is large, hence I can conclusively say that thepo

Question 3
To evaluate if this is a profitable strategy, is to look at the expectancy whether it is positive. To assess robustness, we can look at the maximum drawdown, as a larger drawdown means less robustness. Strategy can also be tested outside this 7 stocks to see if it will yield positive profits with this same strategy. 

Question 4
A suggestion I have would be to incorporate the Sell signal into the strategy by conducting a comparative analysis. Specifically, you could assess whether it is more profitable to hold the shares or sell them when a Sell signal is triggered. This involves evaluating the performance of the portfolio both with and without executing the Sell signals, to determine which approach yields better long-term results. By doing this, you can identify whether reacting to Sell signals is beneficial, or if it's better to let the positions run based on other conditions like trends or technical indicators. This can increase the cumulative returns at the end of the period assessed. 

Question 5
Strategy Overview:
This strategy is built on predefined signals indicating when to enter or exit stock positions. It incorporates fundamental considerations (e.g., dividend dates), technical signals (Buy/Sell indicators), and risk controls (e.g., stop-loss). The strategy aims to capture gains while protecting capital through systematic rebalancing and loss minimization.

Entry Criteria:
Enter a position only when signal = "Buy" is generated, indicating favorable conditions based on either technical indicators or model forecasts.

Exit Criteria:
Exit a position when Signal = "Sell", and current stock price meets or exceeds a predefined profit margin threshold (e.g., 5–10%).
Dividend-Aware Exit - If a stock is nearing its ex-dividend date, and a Sell signal is generated while meeting the profit threshold, consider exiting to avoid short-term price drops typically associated with dividend payouts.
Time-Based Exit - Consider selling after a fixed holding period (e.g., 20 trading days) if neither profit nor loss thresholds are met, to reduce opportunity cost.

Risk Management:
Stop-Loss - Exit the position immediately if daily returns are negative for 3 consecutive trading days to cap potential downside.
Rebalancing - Conduct portfolio rebalancing monthly or quarterly to avoid overexposure to outperforming stocks and maintain diversification.
Use equal weighting or risk-adjusted weights depending on risk appetite.
Position Sizing - Allocate a fixed percentage of capital per trade (e.g., 5%) to reduce the impact of individual losses.

Performance Metric:
Expectancy
Cumulative returns over time
Sharpe ratio to assess risk-adjusted returns
Maximum drawdown to evaluate downside risk